In [1]:
import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, punkt
from gensim.models import KeyedVectors

#setting up libraries
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

In [2]:
df = pd.read_pickle("/home/kno5cac/git/publicrd/data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")

In [3]:
df = df[~df.DEPARTMENT.isnull()]

In [36]:
df_nsf = df.loc[df['DEPARTMENT'] == 'HHS']
df_nsf = df_nsf.iloc[0:10000,]

In [37]:
df_nasa = df.loc[df['DEPARTMENT'] == 'NSF']
df_nasa = df_nasa.iloc[0:10000,]

In [38]:
frames = [df_nsf, df_nasa]
df_split = pd.concat(frames)
#df_split.head()

In [39]:
df_split= df_split.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [40]:
data = df_split['ABSTRACT']

In [41]:
#print(data)

In [42]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
tagged_df = pd.DataFrame(tagged_data)

In [43]:
#print(tagged_df)

In [44]:
df_split['words'] = tagged_df['words']
df_split['tags'] = tagged_df['tags']
df_split.head()

,index,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens,words,tags
0,13120,150681,DESCRIPTION (provided by applicant): The use ...,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,"[description, (, provided, by, applicant, ), :...",[0]
1,13121,150683,DESCRIPTION (provided by applicant): The patho...,5-(6)-carboxyfluorescein diacetate succinimidy...,"DOUBLE NEGATIVE, CD4-CD8- T CELLS IN HUMAN LEI...",HHS,NIH,NIAID,1R03AI066253-01A1,9/30/2008,9/29/2011,"GOLLOB, KENNETH J",NaN,NaN,899644116,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BELO HORIZONTE,NaN,31270-901,BRAZIL,9/30/2008,9/29/2009,93.855,2008,54000.0,NaN,1,1,54000.0,1,pathogen leishmania protozoan_parasite cause w...,"[description, (, provided, by, applicant, ), :...",[1]
2,13122,150684,DESCRIPTION (provided by applicant): \t In thi...,Adopted; AIDS clinical trial group; AIDS/HIV p...,SAO PAULO CLINICAL TRIALS UNIT,HHS,NIH,NIAID,1U01AI069420-01,1/1/2008,12/31/2013,"KALLAS, ESPER GEORGES",NaN,NaN,914608039,FEDERAL UNIVERSITY OF SAO PAULO,SAO PAULO,NaN,04037-003,BRAZIL,1/1/2008,12/31/2008,93.855,2008,493116.0,NaN,1,1,493116.0,1,application propose constitution_sao_paulo cli...,"[description, (, provided, by, applicant, ), :...",[2]
3,13123,150685,DESCRIPTION (provided by applicant): \t The O...,AIDS clinical trial group; AIDS prevention; ba...,THE FIOCRUZ THERAPEUTIC AND PREVENTION HIV/AID...,HHS,NIH,NIAID,1U01AI069476-01,1/15/2008,12/31/2013,"GRINSZTEJN, BEATRIZ",NaN,NaN,899294177,FUNDACAO OSWALDO CRUZ,RIO DE JANEIRO,NaN,21040-360,BRAZIL,1/15/2008,12/31/2008,93.855,2008,3194471.0,NaN,1,1,3194471.0,1,oswaldo_cruz foundation fiocruz therapeutic pr...,"[description, (, provided, by, applicant, ), :...",[3]
4,13124,150686,DESCRIPTION (provided by applicant): \t Fe...,AIDS prevention; AIDS/HIV problem; American; b...,CLINICAL TRIAL UNIT FOR HIV PREVENTION AND MIC...,HHS,NIH,NIAID,1U01AI069480-01,9/8/2008,11/30/2013,"MAYER, KENNETH H",NaN,07,072366156,FENWAY COMMUNITY HEALTH CENTER,BOSTON,MA,022154302,UNITED STATES,9/8/2008,11/30/2008,93.855,2008,637625.0,NaN,1,1,637625.0,1,fenway community health fch_boston community h...,"[description, (, provided, by, applicant, ), :...",[4]


In [45]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, epochs=100) #min_count=2 - min number of times it shows up
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")

In [46]:
model = Doc2Vec.load("d2v.model")

In [47]:
vector = []
for i in range(0, len(df_split)):
    vectorthis = (model.dv[df_split['tags'][i]]).tolist()
    vector.append(vectorthis)

In [48]:
mat = []
for i in vector:
    #dont use this loop
    for x in i:
        mat.append(x)

In [49]:
training = pd.DataFrame(mat)

In [50]:
trainingnp = training.to_numpy()

In [51]:
training_data = df_split.sample(frac=0.85, random_state=25)
testing_data = df_split.drop(training_data.index)
#print(training_data)

In [52]:
train_id = training_data.index
test_id = testing_data.index

In [53]:
#print(test_id)

In [54]:
x_train = trainingnp[train_id]
x_test = trainingnp[test_id]

In [55]:
df_split['index'] = df_split.index
y_test = df_split.loc[df_split["index"].isin(test_id)]
y_test = y_test['DEPARTMENT']

In [56]:
df_split['index'] = df_split.index
y_train = df_split.loc[df_split["index"].isin(train_id)]
y_train = y_train['DEPARTMENT']

In [57]:
import math

In [58]:
neighbors = int(math.sqrt(len(df_split.index)))

In [59]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3)
classifier.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [60]:
y_pred = classifier.predict(x_test)

In [61]:
import scipy.sparse

In [62]:
predictions = df_split.loc[test_id,]
predictions['PREDICTIONS'] = y_pred

In [63]:
predictions = predictions[["index","DEPARTMENT", "PREDICTIONS","ABSTRACT"]]

In [64]:
predictions 

,index,DEPARTMENT,PREDICTIONS,ABSTRACT
14,14,HHS,NSF,DESCRIPTION (provided by applicant): Primary...
32,32,HHS,HHS,DESCRIPTION (provided by applicant): The need ...
62,62,HHS,HHS,DESCRIPTION (provided by applicant): Children ...
71,71,HHS,NSF,DESCRIPTION (provided by applicant): Cryptoco...
89,89,HHS,HHS,The goals of this proposal are to test the hyp...
...,...,...,...,...
19982,19982,NSF,HHS,The goal of this project is to study aerosol-w...
19986,19986,NSF,NSF,In this project the PI proposes to study discr...
19990,19990,NSF,HHS,Microfinance has spread very rapidly in the la...
19994,19994,NSF,NSF,Project Title: Effect of Tax Reform on the Siz...


In [65]:
predictions['PREDICTIONS'].value_counts()

NSF    1546
HHS    1454
Name: PREDICTIONS, dtype: int64

In [66]:
predictions['DEPARTMENT'].value_counts()

HHS    1520
NSF    1480
Name: DEPARTMENT, dtype: int64

In [67]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[705 815]
 [749 731]]
              precision    recall  f1-score   support

         HHS       0.48      0.46      0.47      1520
         NSF       0.47      0.49      0.48      1480

    accuracy                           0.48      3000
   macro avg       0.48      0.48      0.48      3000
weighted avg       0.48      0.48      0.48      3000



In [68]:
error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))

In [69]:
#https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/